In [1]:
# System deps for PDF→image and LaTeX compilation
!apt-get update -qq
!apt-get install -y -qq \
    poppler-utils \
    texlive-latex-base \
    texlive-latex-extra \
    texlive-pictures

# Python libraries
!pip install --upgrade --quiet \
    torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117 \
    datasets transformers peft accelerate bitsandbytes \
    torchmetrics nltk pdf2image evaluate pillow \
    qwen-vl-utils \
    evaluate

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
Preconfiguring packages ...
Selecting previously unselected package fonts-droid-fallback.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../00-fonts-droid-fallback_1%3a6.0.1r16-1.1build1_all.deb ...
Unpacking fonts-droid-fallback (1:6.0.1r16-1.1build1) ...
Selecting previously unselected package fonts-lato.
Preparing to unpack .../01-fonts-lato_2.0-2.1_all.deb ...
Unpacking fonts-lato (2.0-2.1) ...
Selecting previously unselected package poppler-data.
Preparing to unpack .../02-poppler-data_0.4.11-1_all.deb ...
Unpacking poppler-data (0.4.11-1) ...
Selecting previously unselected package tex-common.
Preparing to unpack .../03-tex-common_6.17_all.deb ...
Unpacking tex-common (6.17) ...
Selecting previously unselect

In [2]:
import torch
import torch.nn as nn
import os
import evaluate
import csv
import time
from tqdm import tqdm
from copy import deepcopy
from torch.utils.data import DataLoader, Dataset
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration
from peft import LoraConfig, TaskType, get_peft_model
from datasets import load_dataset
from qwen_vl_utils import process_vision_info
from tqdm.auto import tqdm
from torch.amp import autocast, GradScaler

# Load model + LoRA

In [3]:
from transformers import AutoProcessor, Qwen2VLForConditionalGeneration, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

cache_dir = "/content/cache"
os.makedirs(cache_dir, exist_ok=True)
"""
base = Qwen2VLForConditionalGeneration.from_pretrained(
  "Qwen/Qwen2-VL-7B-Instruct",
  torch_dtype=torch.float16,
  device_map="auto",
  cache_dir=cache_dir
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

tokenizer = processor.tokenizer

lora_cfg = LoraConfig(
  r=8, lora_alpha=16,
  target_modules=["attn.qkv","self_attn.q_proj","self_attn.v_proj"],
  lora_dropout=0.1, bias="none",
  task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(base, lora_cfg)

# save memory during backprop
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
"""
# 1) your processor
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct",
                                          cache_dir="/content/cache")

# 2) define 4-bit quant config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",            # best quality/size tradeoff
    bnb_4bit_compute_dtype=torch.float16, # perform matmuls in FP16
    bnb_4bit_use_double_quant=True,
)

# 3) load base model in 4-bit, auto‐offloaded
base = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",    # accelerate will split 4-bit weights between CPU & GPU
    cache_dir="/content/cache",
)

# 4) freeze & prepare for PEFT
base = prepare_model_for_kbit_training(base)

# 5) attach your LoRA
lora_cfg = LoraConfig(
    r=8, lora_alpha=16,
    target_modules=["attn.qkv","self_attn.q_proj","self_attn.v_proj"],
    lora_dropout=0.1, bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora_cfg)

tokenizer = processor.tokenizer

model = get_peft_model(base, lora_cfg)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


# Training-friendly wrapper

In [4]:
class TikZGenModel(torch.nn.Module):
    def __init__(self, model, processor):
        super().__init__()
        self.model = model
        self.processor = processor

    def forward(self,
                pixel_values=None,
                input_ids=None,
                attention_mask=None,
                image_grid_thw=None,
                labels=None):
        # TRAINING mode: return loss+logits if labels provided
        if labels is not None:
            return self.model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                image_grid_thw=image_grid_thw,
                labels=labels,
            )
        # INFERENCE mode: generate text
        generated = self.model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            image_grid_thw=image_grid_thw,
            max_new_tokens=512,
        )
        decoded = self.processor.batch_decode(
            generated, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )
        return decoded

tikz_model = TikZGenModel(model, processor)

# Preprocessing helper

In [5]:
from copy import deepcopy
from PIL import Image as PILImage

def preprocess_dataset(raw_split,
                       template,
                       dummy_pv=None,
                       dummy_grid=None):
    """
    - Always inject ex['image'] into any content item with type=='image'.
    - Always inject ex['caption'] into any content item with type=='text'.
    - If the template truly has no image (i.e. text-only), use the dummies.
    """
    processed = []
    for ex in tqdm(raw_split, desc="preprocessing"):
        # 1) copy template
        msg = deepcopy(template)

        # 2) inject image & text
        for content in msg[0]["content"]:
            if content["type"] == "image":
                content["image"] = ex["image"]          # ← guaranteed to exist
            if content["type"] == "text":
                content["text"] = content["text"].format(ex["caption"])

        # 3) build the chat string
        chat = processor.apply_chat_template(
            msg, tokenize=False, add_generation_prompt=True
        )

        # 4) decide if we have a real image to load
        #    (we know we do for default & image-only; for text-only, template has no image entry)
        has_image = any(c["type"] == "image" for c in msg[0]["content"])
        if has_image:
            imgs, vids = process_vision_info(msg)
            inputs = processor(
                text=[chat],
                images=imgs,
                videos=vids,
                padding=True,
                return_tensors="pt",
            )
        else:
            # text-only mode: no vision inputs
            inputs = processor(
                text=[chat],
                padding=True,
                return_tensors="pt",
            )

        # 5) if we're in text-only, fill in dummy vision tensors
        if "pixel_values" not in inputs:
            assert dummy_pv is not None and dummy_grid is not None
            inputs["pixel_values"]   = dummy_pv
            inputs["image_grid_thw"] = dummy_grid

        # 6) collect
        processed.append({
            "pixel_values":   inputs["pixel_values"].squeeze(0),
            "image_grid_thw": inputs["image_grid_thw"].squeeze(0),
            "input_ids":      inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "labels":         inputs["input_ids"].squeeze(0),
        })

    return processed

# Message templates

In [6]:
default_msg = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": None},
            {
                "type": "text",
                "text": (
                    "Using the image and its caption below, provide tikz code to generate it. "
                    "Image caption: {}"
                ),
            },
        ],
    }
]

image_only_msg = deepcopy(default_msg)
image_only_msg[0]["content"][1]["text"] = "Provide tikz code to recreate the image."

text_only_msg = [
    {
        "role": "user",
        "content": [
            default_msg[0]["content"][1]  # just the caption‐based text prompt
        ],
    }
]

# Load & split dataset

In [7]:
# load & initial train/test split (90/10)
full_data = load_dataset("nllg/datikz-v3", split="train[:2000]").shuffle(seed=42)
split1 = full_data.train_test_split(test_size=0.10, seed=42)
train_plus, test_raw = split1["train"], split1["test"]

# further split train_plus into train/val (80/10 overall)
split2 = train_plus.train_test_split(test_size=0.1111, seed=42)
train_raw, val_raw = split2["train"], split2["test"]

print(f"Sizes → train: {len(train_raw)}, val: {len(val_raw)}, test: {len(test_raw)}")

README.md:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

train-00000-of-00014.parquet:   0%|          | 0.00/412M [00:00<?, ?B/s]

train-00001-of-00014.parquet:   0%|          | 0.00/442M [00:00<?, ?B/s]

train-00002-of-00014.parquet:   0%|          | 0.00/467M [00:00<?, ?B/s]

train-00003-of-00014.parquet:   0%|          | 0.00/461M [00:00<?, ?B/s]

...

train-00013-of-00014.parquet:   0%|          | 0.00/441M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/33.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/145366 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/542 [00:00<?, ? examples/s]

Sizes → train: 1600, val: 200, test: 200


# Phase 1: preprocess image-only & get dummies

# Build three train splits

In [8]:
train_image_only = preprocess_dataset(
    train_raw,
    image_only_msg
)

# grab one real sample so we have the right tensor shapes
sample = train_image_only[0]
DUMMY_PV   = sample["pixel_values"].unsqueeze(0)
DUMMY_GRID = sample["image_grid_thw"].unsqueeze(0)


preprocessing:   0%|          | 0/1600 [00:00<?, ?it/s]

# Phase 2 & 3: preprocess text-only and both

In [9]:
train_text_only = preprocess_dataset(
    train_raw,
    text_only_msg,
    dummy_pv   = DUMMY_PV,
    dummy_grid = DUMMY_GRID,
)

train_both = preprocess_dataset(
    train_raw,
    default_msg,
    dummy_pv   = DUMMY_PV,     # these won’t be used, since default_msg has an image
    dummy_grid = DUMMY_GRID,
)

preprocessing:   0%|          | 0/1600 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/1600 [00:00<?, ?it/s]

# Build DataLoaders

In [10]:
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TikzDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self):     return len(self.data)
    def __getitem__(self, i): return self.data[i]


def collate_proc(batch):
    # batch is list of dicts
    pv   = torch.stack([x["pixel_values"]   for x in batch])
    grid = torch.stack([x["image_grid_thw"]  for x in batch])
    ids  = pad_sequence([x["input_ids"]      for x in batch],
                        batch_first=True, padding_value=tokenizer.pad_token_id)
    attm = pad_sequence([x["attention_mask"] for x in batch],
                        batch_first=True, padding_value=0)
    lbls = pad_sequence([x["labels"]         for x in batch],
                        batch_first=True, padding_value=-100)

    return {
      "pixel_values":   pv.to(device),
      "image_grid_thw": grid.to(device),
      "input_ids":      ids.to(device),
      "attention_mask": attm.to(device),
      "labels":         lbls.to(device),
    }


loaders = [
  ("Image-Only", DataLoader(train_image_only,  batch_size=1, shuffle=True, collate_fn=collate_proc)),
  ("Text-Only",  DataLoader(train_text_only,  batch_size=1, shuffle=True, collate_fn=collate_proc)),
  ("Both",       DataLoader(train_both, batch_size=1, shuffle=True, collate_fn=collate_proc)),
]


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Preprocess and dataloadoser validation and test

In [11]:
val_image_only = preprocess_dataset(val_raw,  image_only_msg)
val_text_only = preprocess_dataset(val_raw,  text_only_msg,  dummy_pv=DUMMY_PV, dummy_grid=DUMMY_GRID)
val_both = preprocess_dataset(val_raw,  default_msg,    dummy_pv=DUMMY_PV, dummy_grid=DUMMY_GRID)

val_loaders = [
  ("Image-Only", DataLoader(val_image_only, batch_size=1, collate_fn=collate_proc)),
  ("Text-Only",  DataLoader(val_text_only,  batch_size=1, collate_fn=collate_proc)),
  ("Both",       DataLoader(val_both,       batch_size=1, collate_fn=collate_proc)),
]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

In [12]:
test_codes = [ex["code"] for ex in test_raw]
test_image_only = preprocess_dataset(
    test_raw,
    image_only_msg
)
test_text_only = preprocess_dataset(
    test_raw,
    text_only_msg,
    dummy_pv   = DUMMY_PV,
    dummy_grid = DUMMY_GRID,
)
test_both = preprocess_dataset(
    test_raw,
    default_msg,
    dummy_pv   = DUMMY_PV,
    dummy_grid = DUMMY_GRID,
)

test_loaders = [
    ("Image-Only", DataLoader(test_image_only, batch_size=1, shuffle=False, collate_fn=collate_proc)),
    ("Text-Only",  DataLoader(test_text_only,  batch_size=1, shuffle=False, collate_fn=collate_proc)),
    ("Both",       DataLoader(test_both,       batch_size=1, shuffle=False, collate_fn=collate_proc)),
]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

preprocessing:   0%|          | 0/200 [00:00<?, ?it/s]

# Continual three-phase training

In [16]:
scaler   = GradScaler()
save_dir = "/content/checkpoints"
os.makedirs(save_dir, exist_ok=True)

# CSV log for phase metrics
log_path = os.path.join(save_dir, "phase_metrics.csv")
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "phase", "modality",
        "train_time_s", "val_time_s",
        "train_mem_peak_GB", "val_mem_peak_GB",
        "val_loss"
    ])

# three‐phase train + validation
for phase_idx, ((name, train_loader), (_, val_loader)) in enumerate(zip(loaders, val_loaders), 1):
    # training
    print(f"\n=== Phase {phase_idx}: {name} TRAIN ===")
    model.train()

    torch.cuda.reset_peak_memory_stats()
    train_start = time.time()

    for batch in tqdm(train_loader, desc=f"Train Phase {phase_idx}: {name}", total=len(train_loader)):
        optimizer.zero_grad()
        if name != "Text-Only":
            batch["pixel_values"] = batch["pixel_values"].requires_grad_()
        else:
            batch.pop("pixel_values",   None)
            batch.pop("image_grid_thw",  None)

        with autocast("cuda"):
            out = model(**batch)

        scaler.scale(out.loss).backward()
        scaler.step(optimizer)
        scaler.update()

    train_time = time.time() - train_start
    train_mem_peak = torch.cuda.max_memory_allocated() / 1e9

    # save checkpoint
    ckpt = os.path.join(save_dir, f"tikz_phase{phase_idx}.pt")
    torch.save(model.state_dict(), ckpt)
    print(f"Saved model to {ckpt}")

    # validation
    print(f"=== Phase {phase_idx}: {name} VALIDATION ===")
    model.eval()

    torch.cuda.reset_peak_memory_stats()
    val_start = time.time()
    val_losses = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Val Phase {phase_idx}: {name}", total=len(val_loader)):
            if name == "Text-Only":
                batch.pop("pixel_values",   None)
                batch.pop("image_grid_thw",  None)

            batch = {k: v.to(device) for k, v in batch.items()}
            out   = model(**batch)
            val_losses.append(out.loss.item())

    val_time      = time.time() - val_start
    val_mem_peak  = torch.cuda.max_memory_allocated() / 1e9  # in GB
    avg_loss = sum(val_losses) / len(val_losses)

    print(f"→ Phase {phase_idx}/{name}  train_time={train_time:.1f}s  "
          f"train_mem_peak={train_mem_peak:.1f}GB  "
          f"val_time={val_time:.1f}s  val_mem_peak={val_mem_peak:.1f}GB  "
          f"val_loss={avg_loss:.4f}")

    # log to CSV
    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            phase_idx, name,
            f"{train_time:.1f}", f"{val_time:.1f}",
            f"{train_mem_peak:.2f}", f"{val_mem_peak:.2f}",
            f"{avg_loss:.4f}"
        ])

print(f"\nAll done! Phase metrics logged to: {log_path}")


=== Phase 1: Image-Only TRAIN ===


Train Phase 1: Image-Only:   0%|          | 0/1600 [00:00<?, ?it/s]

Saved model to /content/checkpoints/tikz_phase1.pt
=== Phase 1: Image-Only VALIDATION ===


Val Phase 1: Image-Only:   0%|          | 0/200 [00:00<?, ?it/s]

→ Phase 1/Image-Only  train_time=687.5s  train_mem_peak=10.2GB  val_time=80.3s  val_mem_peak=9.0GB  val_loss=7.0608

=== Phase 2: Text-Only TRAIN ===


...

→ Phase 2/Text-Only  train_time=660.6s  train_mem_peak=30.2GB  val_time=83.7s  val_mem_peak=12.9GB  val_loss=1.3486

=== Phase 3: Both TRAIN ===


Train Phase 3: Both:   0%|          | 0/1600 [00:00<?, ?it/s]

Saved model to /content/checkpoints/tikz_phase3.pt
=== Phase 3: Both VALIDATION ===


Val Phase 3: Both:   0%|          | 0/200 [00:00<?, ?it/s]

→ Phase 3/Both  train_time=887.8s  train_mem_peak=31.0GB  val_time=142.3s  val_mem_peak=13.4GB  val_loss=4.2915

All done! Phase metrics logged to: /content/checkpoints/phase_metrics.csv


In [17]:
name, test_loader = test_loaders[2]   # (“Both”, …)
model.eval()

preds, refs = [], []
for idx, batch in enumerate(tqdm(test_loader, desc="Evaluating test", total=len(test_loader))):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        gen_ids = model.generate(
            pixel_values   = batch.get("pixel_values", None),
            image_grid_thw = batch.get("image_grid_thw", None),
            input_ids      = batch["input_ids"],
            attention_mask = batch["attention_mask"],
            max_new_tokens = 512,
        )
    preds.extend(processor.batch_decode(
        gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    ))

    # pull the corresponding raw code string
    refs.append(test_codes[idx])

Evaluating test:   0%|          | 0/200 [00:00<?, ?it/s]

In [18]:
bleu = evaluate.load("bleu")
score = bleu.compute(predictions=preds,
                     references=[[r] for r in refs])["bleu"]
print(f"BLEU: {score*100:.2f}")

BLEU: 23.03


In [21]:
!pip install --upgrade crystalbleu

In [25]:
# 2) Import
from crystalbleu import corpus_bleu

# 3) Prepare your references exactly the same way you did for BLEU:
#    a list of 1-element lists of strings
references_cb = [[r] for r in refs]

# 4) Call CrystalBLEU
#    - `ignoring={}` means “don’t drop any trivial tokens”
#    - `n=4` uses up to 4-gram overlap (the default)
cb_score = corpus_bleu(
    references_cb,   # List[List[str]]
    preds,     # List[str]
    ignoring={},     # or supply a dict of common tokens to ignore
)

print(f"CrystalBLEU: {cb_score*100:.2f}")

CrystalBLEU: 26.18


In [33]:
print(preds[100])

system
You are a helpful assistant.
user
Using the image and its caption below, provide tikz code to generate it. Image caption: Incorrect plot with pgfplots and gnuplot

I'm attempting to plot atanh(x) although there seems to be a giant straight line linking the top and bottom points. Am I doing something wrong?
\documentclass{standalone}
\usepackage{pgfplots}
\begin{document}
\begin{tikzpicture}
  \begin{axis}[no markers,samples=1001]
    \addplot gnuplot{atanh(x)};
  \end{axis}
\end{tikzpicture}
\end{document}
assistant
The issue you're encountering is due to the fact that the function atanh(x) is undefined at x = 1 and x = -1, which are the points where the plot seems to be connecting. To fix this, you can add a small epsilon value to the x values to avoid these undefined points. Here's how you can modify your code:

```latex
\documentclass{standalone}
\usepackage{pgfplots}
\begin{document}
\begin{tikzpicture}
  \begin{axis}[no markers,samples=1001]
    \addplot gnuplot{atanh(x+1e-

In [34]:
print(refs[100])

\documentclass{standalone}
\usepackage{tkz-fct}
\begin{document}
\begin{tikzpicture}
 \tkzInit[xmin=-1,xmax=1,xstep=.2,
          ymin=-5,ymax=5,ystep=1]
 \tkzGrid[color=brown,sub,subxstep=.1](-1,-5)(1,5)
 \tkzAxeXY
 \tkzFct[color=red,samples=1001,domain = -1:1]%
    {atanh(\x)}  
\end{tikzpicture}
\end{document}


In [35]:
!sudo apt-get update
!sudo apt-get install -y texlive-latex-base texlive-latex-extra texlive-pictures poppler-utils!
!pip install pdf2image

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 129 kB in 1s (143 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done

In [36]:
import os
import subprocess
import tempfile
from pdf2image import convert_from_path
from PIL import Image

def compile_tikz_to_image(tikz_code: str, dpi: int = 300) -> Image.Image:
    """
    Compile a TikZ code snippet or full .tex document to a PIL Image.

    If `tikz_code` starts with a `\documentclass` declaration, it's treated as a
    complete LaTeX document. Otherwise, it is wrapped in a minimal standalone TikZ
    document. The function runs `pdflatex` to generate a PDF and converts the first
    page to a PNG image using `pdf2image`.
    """
    # Determine if this is a full document or snippet
    if tikz_code.lstrip().startswith(r"\documentclass"):
        full_tex = tikz_code
    else:
        full_tex = (
            "\\documentclass[tikz,border=2pt]{standalone}\n"
            "\\usepackage{tikz}\n"
            "\\begin{document}\n"
            f"{tikz_code}\n"
            "\\end{document}"
        )
    try:
        with tempfile.TemporaryDirectory() as tmpdir:
            tex_path = os.path.join(tmpdir, "compiled.tex")
            with open(tex_path, "w") as f:
                f.write(full_tex)

            # Compile to PDF silently
            subprocess.run(
                ["pdflatex", "-halt-on-error", tex_path],
                cwd=tmpdir,
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL,
                check=True
            )

            # Convert PDF -> PIL Image
            pdf_path = os.path.join(tmpdir, "compiled.pdf")
            pages = convert_from_path(pdf_path, dpi=dpi)
            return pages[0]
    except subprocess.CalledProcessError as e:
        print(f"[compile warning] pdflatex failed: {e}")
    except Exception as e:
        print(f"[compile warning] conversion error: {e}")

    # On failure, return None
    return None

In [39]:
!pip install -q transformers

In [49]:
from transformers import CLIPProcessor, CLIPModel
import torch
from tqdm.auto import tqdm

# 1) Compile & track success
compiled_figs = []
success_flags = []

for code in tqdm(preds, desc="Compiling TikZ to images"):
    img = compile_tikz_to_image(code, dpi=200)
    if img is not None:
        compiled_figs.append(img)
        success_flags.append(True)
    else:
        compiled_figs.append(Image.new("RGB", (256, 256), "white"))
        success_flags.append(False)

# 2) Filter to only successful ones
filtered_preds = [p for p, ok in zip(preds, success_flags) if ok]
filtered_imgs  = [img for img, ok in zip(compiled_figs, success_flags) if ok]

print(f"Using {len(filtered_preds)}/{len(preds)} compiled samples for CLIPScore")

# 3) Load CLIP
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip      = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)

# 4) Compute CLIPScore on filtered set
all_sims = []
batch_size = 8

for i in tqdm(range(0, len(filtered_imgs), batch_size), desc="CLIP on successes"):
    batch_imgs = filtered_imgs[i : i+batch_size]
    batch_txts = filtered_preds[i : i+batch_size]

    inputs = processor(
        text=batch_txts,
        images=batch_imgs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=processor.tokenizer.model_max_length,
    ).to(device)

    with torch.no_grad():
        out = clip(**inputs)
        img_embeds  = out.image_embeds
        text_embeds = out.text_embeds

    img_norm = img_embeds  / img_embeds.norm(p=2, dim=1, keepdim=True)
    txt_norm = text_embeds / text_embeds.norm(p=2, dim=1, keepdim=True)
    sims     = (img_norm * txt_norm).sum(dim=1)
    all_sims.append(sims.cpu())

all_sims = torch.cat(all_sims)
mean_sim = all_sims.mean().item()
print(f"Mean CLIPScore on compiled images: {mean_sim*100:.2f}%")


Compiling TikZ to images:   0%|          | 0/200 [00:00<?, ?it/s]

[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmppl3d0wiu/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmp9sepxs9s/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmpwcgsfl6p/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmpndajtwsd/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmpgakkymdg/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmp20qg7w6v/compiled.tex']' returned non-zero exit status 1.
[compile warning] pdflatex failed: Command '['pdflatex', '-halt-on-error', '/tmp/tmpfpxl9hmd/compiled.tex']' returned non-zero exit status 1.
[compi

CLIP on successes:   0%|          | 0/2 [00:00<?, ?it/s]

Mean CLIPScore on compiled images: 22.58%


In [46]:
import csv
import os

# 1. Ensure preds and refs are the same length
assert len(preds) == len(refs), "preds and refs must align"

# 2. Path for output CSV
out_path = "/content/checkpoints/predictions_vs_references.csv"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

# 3. Write CSV with header
with open(out_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["prediction", "reference"])
    for pred, ref in zip(preds, refs):
        writer.writerow([pred, ref])

print(f"Wrote {len(preds)} rows to {out_path}")

Wrote 200 rows to /content/checkpoints/predictions_vs_references.csv
